In [62]:
import os
import PyPDF2
import pandas as pd

import numpy as np
import nltk 
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize


# Function to extract PDF content
def extract_pdf_content(pdf_file):
    text = ''
    pdf_reader = PyPDF2.PdfFileReader(pdf_file)
    for page_num in range(pdf_reader.numPages):
        page = pdf_reader.getPage(page_num)
        text += page.extractText()
    return text

# Path to the folder containing PDF files
pdf_folder = 'C:\\Users\\sitas\\OneDrive\\Desktop\\Python\\Data mesh\\dataset'

# Create an empty DataFrame to store the data
pdf_data = pd.DataFrame(columns=['pdf_name', 'pdf_content'])
# Loop through the PDF files in the folder
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        pdf_content = extract_pdf_content(pdf_path)
        pdf_data = pdf_data.append({'pdf_name': pdf_file, 'pdf_content': pdf_content}, ignore_index=True)

# Display the DataFrame
pdf_data



,pdf_name,pdf_content
0,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...
1,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...
2,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...
3,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...
4,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...
5,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...
6,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...
7,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...
8,datamesh - Copy (2).pdf,Data Mesh: Best Practices to Avoid the Data Me...
9,datamesh - Copy (3).pdf,Data Mesh: Best Practices to Avoid the Data Me...


In [43]:
'''THis block of code pre processes the data'''# stemmer = SnowballStemmer("english") 
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english")+["Data Mesh","data mesh","data","mesh"])

def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE) #remove white space
    RE_TAGS = re.compile(r"<[^>]+>") #removes tags
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE) #keep only asscii character
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    RE_DATAMESH = re.compile(r"\bdata\s*mesh\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", str(text))
    text = re.sub(RE_ASCII, " ", str(text))
    text = re.sub(RE_SINGLECHAR, " ", str(text))
    text = re.sub(RE_WSPACE, " ", str(text))

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [lemmatizer.lemmatize(w) for w in words_tokens_lower if w not in stop_words]
        
#         words_filtered = [
#             stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
#         ]

    text_clean = " ".join(words_filtered)
    return text_clean
def drop_nan(df):
    
    df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True)
    
    return df
    

In [44]:
drop_nan(pdf_data)
pdf_data['pdf_content'] = pdf_data['pdf_content'].apply(clean_text)


In [45]:
pdf_data

,pdf_name,pdf_content
0,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,adopting principle boost sharing clinical tria...
1,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,adopting principle boost sharing clinical tria...
2,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,adopting principle boost sharing clinical tria...
3,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,adopting principle boost sharing clinical tria...
4,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,approach enabling centric application tactical...
5,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,approach enabling centric application tactical...
6,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,approach enabling centric application tactical...
7,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,approach enabling centric application tactical...
8,datamesh - Copy (2).pdf,best practice avoid mess jan bode ibm germany ...
9,datamesh - Copy (3).pdf,best practice avoid mess jan bode ibm germany ...


In [46]:
from bertopic import BERTopic



topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(pdf_data['pdf_content'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-10-29 17:01:31,316 - BERTopic - Transformed documents to Embeddings
2023-10-29 17:01:33,417 - BERTopic - Reduced dimensionality
2023-10-29 17:01:33,424 - BERTopic - Clustered reduced embeddings


In [47]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,16,-1_organization_domain_product_question


In [51]:
pdf_contents = []
topic_groups = []
top_topic_words = []
dominant_words = []

# Loop through the documents and their corresponding topics
for i, (topic, prob) in enumerate(zip(topics, probs)):
    pdf_contents.append(pdf_data['pdf_content'].iloc[i])  # Extract the original 'pdf_content'
    
    # Extract the topic group (label with highest probability)
    topic_group = topic
    topic_groups.append(topic_group)
    
    # Get the top words for the topic group
    top_words = topic_model.get_topic(topic_group)
    top_topic_words.append(top_words)
    
    # Get the dominant word for the document
    dominant_word = top_words[0][0]  # The most probable word in the top words
    dominant_words.append(dominant_word)

# Create a DataFrame to store the results
result_df = pd.DataFrame({
    'pdf_content': pdf_contents,
    'topic_group': topic_groups,
    'top_topic_words': top_topic_words,
    'dominant_word': dominant_words
})

In [52]:
result_df

,pdf_content,topic_group,top_topic_words,dominant_word
0,adopting principle boost sharing clinical tria...,-1,"[(organization, 0.059329953483435095), (domain...",organization
1,adopting principle boost sharing clinical tria...,-1,"[(organization, 0.059329953483435095), (domain...",organization
2,adopting principle boost sharing clinical tria...,-1,"[(organization, 0.059329953483435095), (domain...",organization
3,adopting principle boost sharing clinical tria...,-1,"[(organization, 0.059329953483435095), (domain...",organization
4,approach enabling centric application tactical...,-1,"[(organization, 0.059329953483435095), (domain...",organization
5,approach enabling centric application tactical...,-1,"[(organization, 0.059329953483435095), (domain...",organization
6,approach enabling centric application tactical...,-1,"[(organization, 0.059329953483435095), (domain...",organization
7,approach enabling centric application tactical...,-1,"[(organization, 0.059329953483435095), (domain...",organization
8,best practice avoid mess jan bode ibm germany ...,-1,"[(organization, 0.059329953483435095), (domain...",organization
9,best practice avoid mess jan bode ibm germany ...,-1,"[(organization, 0.059329953483435095), (domain...",organization


In [67]:
'''Topic modeling'''
def NMF_modeling(df_data, components):
    
    df = df_data.copy()
    model = TfidfVectorizer(stop_words='english')
    word_vector = normalize(model.fit_transform(df['pdf_content']))
    nmf_model = NMF(n_components= components, init='nndsvd')
    nmf_model.fit(word_vector)
    df['topic'] = nmf_model.transform(word_vector).argmax(axis=1)
    
    topic_words = []
    n_words = 5
    
    feature_names = model.get_feature_names_out()
    for topic_idx, topic in enumerate(nmf_model.components_):
        top_words_idx = topic.argsort()[:-n_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        topic_words.append(', '.join(top_words))

    df['topic_words'] = [topic_words[i] for i in df['topic']]
    
    # Transform the topic_words column into a TF-IDF matrix
    tfidf_matrix = model.transform(df['topic_words'])
    dominant_words = []
    for i in range(len(df)):
        row = tfidf_matrix[i]
        dominant_word_idx = row.argmax()
        dominant_word = model.get_feature_names_out()[dominant_word_idx]
        dominant_words.append(dominant_word)

# Add the dominant_word column to the DataFrame
    df['dominant_word'] = dominant_words
    
    return df



In [68]:

result = NMF_modeling(pdf_data, 4)

result

,pdf_name,pdf_content,topic,topic_words,dominant_word
0,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...,1,"data, g12, g1, g9, g10",g1
1,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...,1,"data, g12, g1, g9, g10",g1
2,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...,1,"data, g12, g1, g9, g10",g1
3,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...,1,"data, g12, g1, g9, g10",g1
4,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...,2,"data, tactical, coalition, military, layer",coalition
5,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...,2,"data, tactical, coalition, military, layer",coalition
6,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...,2,"data, tactical, coalition, military, layer",coalition
7,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...,2,"data, tactical, coalition, military, layer",coalition
8,datamesh - Copy (2).pdf,Data Mesh: Best Practices to Avoid the Data Me...,0,"data, mesh, interviewees, organizations, domain",interviewees
9,datamesh - Copy (3).pdf,Data Mesh: Best Practices to Avoid the Data Me...,0,"data, mesh, interviewees, organizations, domain",interviewees
